# [Wip] Cate Example

The objective os this notebook is to build a causal inference model based on the paper by Athey, Imbens 2015 using dataset from cunning 2021.

Specifically we want to:
1. Explain the dataset;
2. Compute the Real ATE;
3. Compute The propensity score & transform Y (used in both next two stages);
4. Estimate the ATE from biased data;
5. Estimate the Cate from biased data. 

In [66]:
# Session Setup
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression, RandomForestRegressor 
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import FloatType

from ds_toolbox.utils import start_local_spark

pd.set_option('display.max_columns', 500)
spark = start_local_spark(max_mem=1, n_cores=1)

# Functions
def read_data(file): 
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)

get_p1 = F.udf(lambda value: value[1].item(), FloatType())

## 1) The dataset

Describe the datasets here

In [33]:
# DataFrame Experimental
dfs_experimental = spark.createDataFrame(read_data('nsw_mixtape.dta'))

#DataFrame with Bias selection
dfs_biased = dfs_experimental.union(spark.createDataFrame(read_data('cps_mixtape.dta')))\
    .withColumn('age2', F.col('age')**2)\
    .withColumn('age3', F.col('age')**3)\
    .withColumn('educ2', F.col('educ')**2)\
    .withColumn('educ_re74', F.col('educ')*F.col('re74'))\
    .withColumn('u74', F.when(F.col('re74')==0, 1).otherwise(0))\
    .withColumn('u75', F.when(F.col('re75')==0, 1).otherwise(0))


dfs_biased.show(5)

+--------------------+-----+----+----+-----+----+----+--------+----+----+-----------------+------+-------+-----+---------+---+---+
|             data_id|treat| age|educ|black|hisp|marr|nodegree|re74|re75|             re78|  age2|   age3|educ2|educ_re74|u74|u75|
+--------------------+-----+----+----+-----+----+----+--------+----+----+-----------------+------+-------+-----+---------+---+---+
|Dehejia-Wahba Sample|  1.0|37.0|11.0|  1.0| 0.0| 1.0|     1.0| 0.0| 0.0|  9930.0458984375|1369.0|50653.0|121.0|      0.0|  1|  1|
|Dehejia-Wahba Sample|  1.0|22.0| 9.0|  0.0| 1.0| 0.0|     1.0| 0.0| 0.0| 3595.89404296875| 484.0|10648.0| 81.0|      0.0|  1|  1|
|Dehejia-Wahba Sample|  1.0|30.0|12.0|  1.0| 0.0| 0.0|     0.0| 0.0| 0.0|   24909.44921875| 900.0|27000.0|144.0|      0.0|  1|  1|
|Dehejia-Wahba Sample|  1.0|27.0|11.0|  1.0| 0.0| 0.0|     1.0| 0.0| 0.0| 7506.14599609375| 729.0|19683.0|121.0|      0.0|  1|  1|
|Dehejia-Wahba Sample|  1.0|33.0| 8.0|  1.0| 0.0| 0.0|     1.0| 0.0| 0.0|289.789886

## 2) The Real ATE
Since this was as random experiment we can easily compute the real average treatment effect to use as a benchmark.

In [34]:
mean0 = dfs_experimental.filter(F.col('treat')==0).select(F.avg('re78')).collect()[0][0]
mean1 = dfs_experimental.filter(F.col('treat')==1).select(F.avg('re78')).collect()[0][0]

print(f'ATE = {round(mean1, 2)} - {round(mean0, 2)} = {round(mean1-mean0, 2)}')

ATE = 6349.14 - 4554.8 = 1794.34


## 3) Computing Propensity Score & Transform Y

In [69]:
features=['age', 'age2', 'age3', 'educ', 'educ2', 'marr', 'nodegree', 'black', 'hisp', 're74', 're75', 'u74', 'u75', 'educ_re74']
assembler = VectorAssembler(inputCols=features, outputCol='features')
pipeline = Pipeline(stages = [assembler, LogisticRegression(labelCol='treat', fitIntercept=True)])
fitted_classifier = pipeline.fit(dfs_biased)


dfs_predicted_with_ps_th = fitted_classifier.transform(dfs_biased)\
    .withColumn('ps', get_p1(F.col('probability')))\
    .withColumn('th', F.col('re78')*(F.col('treat')*F.col('ps'))/(F.col('ps')*(1-F.col('ps'))))

dfs_predicted_with_ps_th.sample(fraction=0.05).orderBy(F.col('treat').desc()).toPandas().drop(columns=['features']).head()


,data_id,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,age2,age3,educ2,educ_re74,u74,u75,rawPrediction,probability,prediction,ps,th
0,Dehejia-Wahba Sample,1.0,25.0,11.0,1.0,0.0,1.0,1.0,0.0,0.0,1574.423950,625.0,15625.0,121.0,0.0,1,1,"[1.3435121024079386, -1.3435121024079386]","[0.79306691210761, 0.20693308789238996]",0.0,0.206933,1985.234715
1,Dehejia-Wahba Sample,1.0,25.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,6181.879883,625.0,15625.0,25.0,0.0,1,1,"[0.36373033285875067, -0.36373033285875067]","[0.5899431415709895, 0.4100568584290105]",0.0,0.410057,10478.771609
2,Dehejia-Wahba Sample,1.0,29.0,8.0,1.0,0.0,0.0,1.0,0.0,0.0,1923.937988,841.0,24389.0,64.0,0.0,1,1,"[-0.1351501830741899, 0.1351501830741899]","[0.4662637895374044, 0.5337362104625956]",1.0,0.533736,4126.286725
3,Dehejia-Wahba Sample,1.0,18.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,2321.106934,324.0,5832.0,144.0,0.0,1,1,"[1.580757579049262, -1.580757579049262]","[0.829311782887994, 0.170688217112006]",0.0,0.170688,2798.835121
4,Dehejia-Wahba Sample,1.0,18.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,4814.626953,324.0,5832.0,121.0,0.0,1,1,"[1.2772102295195165, -1.2772102295195165]","[0.7819745215886744, 0.21802547841132558]",0.0,0.218025,6157.012486


## 4) Estimating ATE from Bieased Data

## 5) Estimating the CATE from Biased Data

Steps
* a) Compute the propensity score os treatment;
* b) Transform the Y variable into Y*;
* c) Predict Y* (normal prediction procedures);
* d) Evaluate step c.

In [70]:
target = 'th'
train, test = dfs_predicted_with_ps_th.select('features', 'th').randomSplit([0.8, 0.2], seed=12345)

spark_classifiers = {
    #'logistic_regression': LinearRegression(labelCol=target),
    #  'decision_tree': DecisionTreeRegressionModel(predictionCol=target),
      'random_forest': RandomForestRegressor(labelCol=target),
    #  'gradient_boosting': GBTRegressionModel(predictionCol=target)
}

for classifier_name, classifier in spark_classifiers.items():
    print(f'{classifier_name}: Starting')
    pipeline = Pipeline(stages = [classifier])
    # Fit no Modelo de Predict nos test
    print(f'{classifier_name}: Fitting Pipeline')
    fitted_classifier = pipeline.fit(train)
    print(f'{classifier_name}: Making Predictions on test data')
    prediction_on_test = fitted_classifier.transform(test)
    break

random_forest: Starting
random_forest: Fitting Pipeline
random_forest: Making Predictions on test data


In [71]:
prediction_on_test.show()

+--------------------+------------------+------------------+
|            features|                th|        prediction|
+--------------------+------------------+------------------+
|(14,[0,1,2,3,4,6,...|13386.756789547962| 634.2559682630944|
|(14,[0,1,2,3,4,7,...|               0.0|2025.1604229086108|
|(14,[0,1,2,3,4,7,...|               0.0|2025.1604229086108|
|(14,[0,1,2,3,4,7,...|               0.0|3748.7449887462244|
|(14,[0,1,2,3,4,7,...|               0.0|  3533.86533323607|
|(14,[0,1,2,3,4,7,...|47712.014965533985|3993.8831559332148|
|(14,[0,1,2,3,4,7,...|               0.0|4553.7725196473475|
|(14,[0,1,2,3,4,11...| 8135.580899398114| 260.0003933968165|
|[17.0,289.0,4913....|               0.0| 1290.459031397421|
|[17.0,289.0,4913....|               0.0| 1290.459031397421|
|[17.0,289.0,4913....|               0.0| 1290.459031397421|
|[18.0,324.0,5832....|               0.0|1351.4932181314593|
|[18.0,324.0,5832....|               0.0|1351.4932181314593|
|[18.0,324.0,5832....|  